In [1]:
from helpers import *
from sketch_transfer import *
from model_1enc import *
import io
import torch.onnx
import onnx
import onnxruntime
%load_ext autoreload
%autoreload 2

In [2]:
hp = HParams()
hp.enc_hidden_size = 256  # 256
hp.dec_hidden_size = 512  # 512
hp.Nmax = 100
hp.save_every = 100
hp.wKL = 0.001
hp.KL_start = 0.01
hp.KL_delta = 0.003
hp.KL_min = 0.0
hp.lr = 0.001
hp.lr_decay = 0.999
hp.style_dim = 7#5
hp.enc_layers = 2
hp.dec_layers = 2
hp.Nz = 2   # latent dimension
hp.Nz_dec = hp.Nz # 1 encoder, 1 decoder
hp.M = 3 # gaussian mixture


#encoder = BiLSTMEncoder(hp, True).cuda()
decoder = LSTMDecoder(hp, False).cuda()
decoder_export = LSTMDecoder_export(hp).cuda()
decoder_init = InitDecoder_export(hp).cuda()

#enc = "models/0203/no latent interpolation/sketch_encoder_sel_0.112513_epoch_700.pth"
dec = "models/0210/sketch_decoder_sel_0.857886_epoch_1000.pth"
#encoder.load_state_dict(torch.load(enc))
state_dict = torch.load(dec)
decoder.load_state_dict(state_dict)
decoder_export.load_state_dict(state_dict)
decoder_init.load_state_dict(state_dict, strict=False)
#encoder.eval()
decoder.eval()
decoder_export.eval()
decoder_init.eval()

InitDecoder_export(
  (fc_hc): Linear(in_features=2, out_features=2048, bias=True)
)

https://pytorch.org/tutorials/advanced/super_resolution_with_onnxruntime.html
Exporting a model in PyTorch works via tracing or scripting. This tutorial will use as an example a model exported by tracing. To export a model, we call the torch.onnx.export() function. This will execute the model, recording a trace of what operators are used to compute the outputs. Because export runs the model, we need to provide an input tensor x. The values in this can be random as long as it is the right type and size. Note that the input size will be fixed in the exported ONNX graph for all the input’s dimensions, unless specified as a dynamic axes. In this example we export the model with an input of batch_size 1, but then specify the first dimension as dynamic in the dynamic_axes parameter in torch.onnx.export(). The exported model will thus accept inputs of size [batch_size, 1, 224, 224] where batch_size can be variable.

To learn more details about PyTorch’s export interface, check out the torch.onnx documentation.

In [19]:
# Input to the model
batch_size = 1
length = 1
input_dim = 2
latent_dim = 2
hidden_dim = 512
with torch.no_grad():
    inputs = torch.randn(length, batch_size, input_dim+latent_dim).cuda()
    z = torch.randn(batch_size, latent_dim).cuda()
    h, c = decoder_init(z)
    decoder_out = decoder_export(inputs, (h, c))

    # Export the model
    torch.onnx.export(decoder_export,               # model being run
                      (inputs, (h, c)),                         # model input (or a tuple for multiple inputs)
                      "decoder.onnx",   # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=11,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['inputs', 'hidden', 'cell'],   # the model's input names
                      output_names = ['pi','mu_x','mu_y','sigma_x','sigma_y','rho_xy','hidden','cell'], # the model's output names
                      dynamic_axes={'inputs' : {0 : 'sequence'}}, verbose=True)
    
    torch.onnx.export(decoder_init,               # model being run
                      z,                         # model input (or a tuple for multiple inputs)
                      "decoder_init.onnx",   # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=11,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['z'],   # the model's input names
                      output_names = ['hidden', 'cell'], verbose=True)

graph(%inputs : Float(1, 1, 4),
      %hidden.1 : Float(2, 1, 512),
      %cell.1 : Float(2, 1, 512),
      %fc_params.weight : Float(18, 512),
      %fc_params.bias : Float(18),
      %214 : Float(1, 2048, 4),
      %215 : Float(1, 2048, 512),
      %216 : Float(1, 4096),
      %234 : Float(1, 2048, 512),
      %235 : Float(1, 2048, 512),
      %236 : Float(1, 4096)):
  %15 : Tensor? = prim::Constant()
  %72 : Tensor = onnx::Constant[value={0}]()
  %73 : Tensor = onnx::Constant[value={0}]()
  %74 : Tensor = onnx::Constant[value={1}]()
  %75 : Tensor = onnx::Slice(%hidden.1, %73, %74, %72)
  %76 : Tensor = onnx::Constant[value={0}]()
  %77 : Tensor = onnx::Constant[value={0}]()
  %78 : Tensor = onnx::Constant[value={1}]()
  %79 : Tensor = onnx::Slice(%cell.1, %77, %78, %76)
  %80 : Tensor, %81 : Tensor, %82 : Tensor = onnx::LSTM[hidden_size=512](%inputs, %214, %215, %216, %15, %75, %79)
  %83 : Tensor = onnx::Squeeze[axes=[1]](%80)
  %140 : Tensor = onnx::Constant[value={0}]()
  %141 :

D:\Dropbox (GaTech)\Dropbox (GaTech)\Research\MS Project\Draw_AI\sketch_transfer.py:156: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pi = F.softmax(pi.transpose(0,1).squeeze()).view(len_out,-1,self.hp.M)


In [4]:
onnx_decoder = onnx.load("decoder.onnx")
onnx_decoder_init = onnx.load("decoder_init.onnx")
for m in [onnx_decoder, onnx_decoder_init]:
    onnx.checker.check_model(m)

In [5]:
init_session = onnxruntime.InferenceSession("decoder_init.onnx")
decode_session = onnxruntime.InferenceSession("decoder.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {init_session.get_inputs()[0].name: to_numpy(z)}
ort_outputs = init_session.run(None, ort_inputs)

torch_out_hc = decoder_init(z)
# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out_hc[0]), ort_outputs[0], rtol=1e-03, atol=1e-05)
np.testing.assert_allclose(to_numpy(torch_out_hc[1]), ort_outputs[1], rtol=1e-03, atol=1e-05)

pi,mu_x,mu_y,sigma_x,sigma_y,rho_xy,hidden,cell = decoder_export(inputs, torch_out_hc[0], torch_out_hc[1])


ort_inputs = {decode_session.get_inputs()[0].name: to_numpy(inputs), 
             decode_session.get_inputs()[1].name: to_numpy(torch_out_hc[0]),
            decode_session.get_inputs()[2].name: to_numpy(torch_out_hc[1])}
ort_outputs = decode_session.run(None, ort_inputs)
np.testing.assert_allclose(to_numpy(hidden), ort_outputs[6], rtol=1e-03, atol=1e-05)
np.testing.assert_allclose(to_numpy(cell), ort_outputs[7], rtol=1e-03, atol=1e-05)